In [1]:
from astroquery.esa.euclid import Euclid
tables = Euclid.load_tables(only_names=True, include_shared_tables=True)
print("Found", len(tables), "tables")
print(*(table.name for table in tables), sep="\n")

# 写入txt文件中
with open("Euclid_tables.txt", "w") as f:
    for table in tables:
        f.write(table.name + "\n")

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]
Found 66 tables
catalogue.mer_catalogue
catalogue.mer_cutouts
catalogue.mer_morphology
catalogue.phz_classification
catalogue.phz_galaxy_sed
catalogue.phz_nir_physical_parameters
catalogue.phz_photo_z
catalogue.phz_physical_parameters
catalogue.phz_qso_physical_parameters
catalogue.phz_star_sed
catalogue.phz_star_template
catalogue.spectro_line_features_catalog_spe_atomic_indices
catalogue.spectro_line_features_catalog_spe_continuum_features
catalogue.spectro_line_features_catalog_spe_line_features_cat
catalogue.spectro_line_features_catalog_spe_molecular_indices
catalogue.spectro_model_catalog_spe_galaxy_models
catalogue.spectro_model_catalog_spe_lines_catalog
catalogue.spectro_model_catalog_spe_qso_models
catalogue.spectro_model_catalog_spe_star_models
catalogue.spectro_zcatalog_spe_classification
catalogue.spectro_zcatalog_spe_galaxy_cand

In [2]:
#example cone search for source NGC6505
from astroquery.esa.euclid import Euclid
from astropy.coordinates import SkyCoord
import astropy.units as u


coord = SkyCoord("17h51m07.4s +65d31m50.8s", frame='icrs')
radius = u.Quantity(0.5, u.deg)
job = Euclid.cone_search(coordinate=coord, radius=radius, table_name="sedm.calibrated_frame", ra_column_name="ra", dec_column_name="dec", columns="*", async_job=True)
# 'https://eas.esac.esa.int/sas-dd/data?file_name=EUC_NIR_W-CAL-IMAGE_Y-2704-0_20240930T182242.317672Z.fits&release=sedm&RETRIEVAL_TYPE=FILE'
cone_results = job.get_results()
print("Found", len(cone_results), "results")
cone_results['creation_date', 'ra', 'dec', 'file_name', 'file_path', 'datalabs_path', 'filter_name', 'dist'][:5]


INFO: Query finished. [astroquery.utils.tap.core]
Found 18 results


creation_date,ra,dec,file_name,file_path,datalabs_path,filter_name,dist
str23,float64,float64,str65,str47,str35,str5,float64
2024-09-30T18:23:19.14,267.99354663,65.60351547,EUC_NIR_W-CAL-IMAGE_Y-2704-0_20240930T182242.317672Z.fits,/euclid/repository_idr/iqr1/Q1_R1/NIR/2704,/data/euclid_q1/Q1_R1/NIR/2704,NIR_Y,0.1141538148715594
2024-10-17T04:30:56.18,267.99354663,65.60351547,EUC_VIS_SWL-DET-002704-00-2-0000000__20241017T042759.344384Z.fits,/euclid/repository_idr/iqr1/Q1_R1/VIS_QUAD/2704,/data/euclid_q1/Q1_R1/VIS_QUAD/2704,VIS,0.1141538148715594
2024-09-30T19:20:24.788,267.99354663,65.60351547,EUC_NIR_W-CAL-IMAGE_H-2704-0_20240930T191946.868701Z.fits,/euclid/repository_idr/iqr1/Q1_R1/NIR/2704,/data/euclid_q1/Q1_R1/NIR/2704,NIR_H,0.1141538148715594
2024-09-30T18:13:05.56,267.99354663,65.60351547,EUC_NIR_W-CAL-IMAGE_J-2704-0_20240930T181230.826554Z.fits,/euclid/repository_idr/iqr1/Q1_R1/NIR/2704,/data/euclid_q1/Q1_R1/NIR/2704,NIR_J,0.1141538148715594
2024-10-17T04:30:39.527,267.99354663,65.60351547,EUC_VIS_SWL-DET-002704-00-1-0000000__20241017T042756.626122Z.fits,/euclid/repository_idr/iqr1/Q1_R1/VIS_QUAD/2704,/data/euclid_q1/Q1_R1/VIS_QUAD/2704,VIS,0.1141538148715594


In [3]:
# 生成数据下载的url
cone_results["url"] = "https://eas.esac.esa.int/sas-dd/data?file_name=" + cone_results["file_name"] + "&release=sedm&RETRIEVAL_TYPE=FILE"
print(cone_results["url"][:5])

                                                                       url                                                                       
-------------------------------------------------------------------------------------------------------------------------------------------------
        https://eas.esac.esa.int/sas-dd/data?file_name=EUC_NIR_W-CAL-IMAGE_Y-2704-0_20240930T182242.317672Z.fits&release=sedm&RETRIEVAL_TYPE=FILE
https://eas.esac.esa.int/sas-dd/data?file_name=EUC_VIS_SWL-DET-002704-00-2-0000000__20241017T042759.344384Z.fits&release=sedm&RETRIEVAL_TYPE=FILE
        https://eas.esac.esa.int/sas-dd/data?file_name=EUC_NIR_W-CAL-IMAGE_H-2704-0_20240930T191946.868701Z.fits&release=sedm&RETRIEVAL_TYPE=FILE
        https://eas.esac.esa.int/sas-dd/data?file_name=EUC_NIR_W-CAL-IMAGE_J-2704-0_20240930T181230.826554Z.fits&release=sedm&RETRIEVAL_TYPE=FILE
https://eas.esac.esa.int/sas-dd/data?file_name=EUC_VIS_SWL-DET-002704-00-1-0000000__20241017T042756.626122Z.fits&release=sed

In [ ]:
import subprocess
import os

def download_with_wget(url, output_path):
    """
    使用wget下载文件，并支持断点续传。
    
    :param url: 文件的URL地址。
    :param output_path: 文件保存的本地路径。
    """
    if not os.path.exists(os.path.dirname(output_path)):
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 使用wget命令下载文件，'-c'参数用于支持断点续传
    result = subprocess.run(['wget', '-c', url, '-O', output_path], capture_output=True, text=True)

    if result.returncode == 0:
        print(f"File {output_path} downloaded successfully.")
    else:
        print(f"Failed to download file {output_path}. Error: {result.stderr}")

output_folder = 'example_outputs/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for example_file_name in cone_results['file_name']:
    print("Getting file:", example_file_name)
    url = "https://eas.esac.esa.int/sas-dd/data?file_name=" + example_file_name + "&release=sedm&RETRIEVAL_TYPE=FILE"
    output_path = os.path.join(output_folder, example_file_name)
    download_with_wget(url, output_path)

Getting file: EUC_NIR_W-CAL-IMAGE_Y-2704-0_20240930T182242.317672Z.fits
